In [14]:
# data processing tools
import string, os 
import pandas as pd
import numpy as np
np.random.seed(42)

# keras module for building LSTM 
import tensorflow as tf
tf.random.set_seed(42)
import tensorflow.keras.utils as ku 
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

# surpress warnings
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

In [15]:
#in-class function
def clean_text(txt):
    txt = "".join(v for v in txt if v not in string.punctuation).lower()
    txt = txt.encode("utf8").decode("ascii",'ignore')
    return txt 


In [16]:
def gather_folder():
    folder = input("write the name of the folder that you would like to use (ex. news_data)")
    return folder

Load data and create corpus

In [17]:
def load_data(filepath):
    data_dir = os.path.join(filepath)

    all_headlines = []
    for filename in os.listdir(data_dir):
        if 'Articles' in filename:
            article_df = pd.read_csv(data_dir + filename)
            all_headlines.extend(list(article_df["headline"].values))
    
    all_headlines = [h for h in all_headlines if h != "Unknown"]

    return all_headlines

def create_corpus(filepath):
    all_headlines = load_data(filepath)
    corpus = [clean_text(x) for x in all_headlines]

    return corpus

In [18]:
filepath = f"../../{gather_folder()}/"

create_corpus(filepath)

print(create_corpus(filepath)[:10])

['my beijing the sacred city', '6 million riders a day 1930s technology', 'seeking a crossborder conference', 'questions for despite the yuck factor leeches are big in russian medicine', 'who is a criminal', 'an antidote to europes populism', 'the cost of a speech', 'degradation of the language', 'on the power of being awful', 'trump garbles pitch on a revised health bill']


In [25]:
def get_sequence_of_tokens(tokenizer, corpus):
    ## convert data to sequence of tokens 
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    return input_sequences

In [19]:
def tokenizing(filepath):
    corpus = create_corpus(filepath)
    tokenizer = Tokenizer()

    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1
    
    inp_sequences = get_sequence_of_tokens(tokenizer, corpus)

    return tokenizer, inp_sequences

In [20]:
def padded_sequences(filepath):
    tokenizer, input_sequences = tokenizing(filepath)
    total_words = len(tokenizer.word_index) + 1
    corpus = create_corpus(filepath)

    # get the length of the longest sequence
    max_sequence_len = max([len(x) for x in input_sequences])
    # make every sequence the length of the longest on
    input_sequences = np.array(pad_sequences(input_sequences, 
                                            maxlen=max_sequence_len, 
                                            padding='pre'))

    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label = ku.to_categorical(label, 
                            num_classes=total_words)
    return predictors, label, max_sequence_len, total_words

In [21]:
def create_model():
    model = create_model(max_sequence_len, total_words)
    model.summary() 

In [27]:
def create_model(filepath):
    predictors, label, max_sequence_len, total_words = padded_sequences(filepath)
    input_len = max_sequence_len - 1
    model = Sequential()
    
    # Add Input Embedding Layer
    model.add(Embedding(total_words, 
                        10, 
                        input_length=input_len))
    
    # Add Hidden Layer 1 - LSTM Layer
    model.add(LSTM(100))
    model.add(Dropout(0.1))
    
    # Add Output Layer
    model.add(Dense(total_words, 
                    activation='softmax'))

    model.compile(loss='categorical_crossentropy', 
                    optimizer='adam')
    
    
    return model

In [32]:
def save_model(filepath):
    predictors, label, max_sequence_len, total_words = padded_sequences(filepath)
    model = create_model(filepath)
    export_path = "../out/"

    history = model.fit(predictors, 
                    label, 
                    epochs=100,
                    batch_size=128, 
                    verbose=1)
    
    tf.keras.saving.save_model(model, export_path, overwrite=True, save_format=None)
    

In [33]:
save_model(filepath)

2023-04-08 23:15:36.621607: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-08 23:15:36.622937: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-08 23:15:36.624137: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Epoch 1/100


2023-04-08 23:15:37.668333: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-08 23:15:37.669950: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-08 23:15:37.671135: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

405/405 [==============================] - 23s 53ms/step - loss: 7.8873
Epoch 2/100
405/405 [==============================] - 21s 52ms/step - loss: 7.4822
Epoch 3/100
405/405 [==============================] - 21s 52ms/step - loss: 7.4001
Epoch 4/100
405/405 [==============================] - 21s 52ms/step - loss: 7.3206
Epoch 5/100
405/405 [==============================] - 21s 53ms/step - loss: 7.2311
Epoch 6/100
405/405 [==============================] - 21s 52ms/step - loss: 7.1260
Epoch 7/100
405/405 [==============================] - 21s 52ms/step - loss: 7.0016
Epoch 8/100
405/405 [==============================] - 21s 53ms/step - loss: 6.8674
Epoch 9/100
405/405 [==============================] - 21s 53ms/step - loss: 6.7347
Epoch 10/100
405/405 [==============================] - 21s 52ms/step - loss: 6.6104
Epoch 11/100
405/405 [==============================] - 21s 53ms/step - loss: 6.4924
Epoch 12/100
405/405 [==============================] - 21s 53ms/step - loss: 6.3794
E

2023-04-08 23:50:55.326802: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-08 23:50:55.327998: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-08 23:50:55.329431: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

INFO:tensorflow:Assets written to: ../out/assets


INFO:tensorflow:Assets written to: ../out/assets
